[View in Colaboratory](https://colab.research.google.com/github/thayumaanavan/TensorFlow/blob/master/Intro_to_TensorFlow.ipynb)

In [0]:
from __future__ import absolute_import,division,print_function

import numpy as np
import tensorflow as tf

### Run TensorBoard in Colab

In [0]:
LOG_DIR = '/tmp/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
! curl http://localhost:6006

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1
! unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [14]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://b2d2aa09.ngrok.io


# Tensorflow Basics

## Tensors

In [5]:
a = tf.constant(4.3)
b = tf.constant(2.4, dtype = tf.float32)
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


##  Graphs

In [0]:
writer = tf.summary.FileWriter('/tmp/log')
writer.add_graph(tf.get_default_graph())

## Session

In [17]:
sess = tf.Session()
print(sess.run(total))

6.7000003


In [18]:
print(sess.run({'ab':(a,b), 'total': total}))

{'ab': (4.3, 2.4), 'total': 6.7000003}


In [21]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.6673131  0.7757058  0.00575542]
[0.88059056 0.7212678  0.5014881 ]
(array([1.5904428, 1.869086 , 1.6414287], dtype=float32), array([2.5904427, 2.869086 , 2.6414287], dtype=float32))


## Feeding

In [25]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(sess.run(z, feed_dict = {x: 4, y: 1.3}))
print(sess.run(z, feed_dict = {x: [1, 2, 3], y:[2., 5, 7]}))

5.3
[ 3.  7. 10.]


## Datasets

In [0]:
my_data = [
    [0, 1],
    [2, 3],
    [4, 5],
    [6, 7]
]

slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

In [37]:
while True:
  try:
    print(sess.run(next_item))  
  except tf.errors.OutOfRangeError:
    print('End of data')
    break

[0 1]
[2 3]
[4 5]
[6 7]
End of data


In [39]:
r = tf.random_normal([10, 3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while(True):
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break

[ 0.27022338  0.39980522 -1.4244442 ]
[-0.6821632  -0.8306161   0.42343733]
[-0.9722504   1.1778765   0.20370676]
[ 1.4144363   0.10453297 -0.3476158 ]
[1.447857   1.0573546  0.85842353]
[-0.7125022 -0.7656749  0.5010435]
[ 0.8720069   0.04987228 -2.2587707 ]
[ 0.33332777 -0.13372426 -0.69012225]
[-3.6631515  -0.68921554 -0.7903427 ]
[0.6301057  0.81646156 1.8055857 ]


## Layers



### Creating Layers

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

### Initializing Layers

In [0]:
init = tf.global_variables_initializer()
sess.run(init)


### Executing Layers

In [50]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[-1.4604723]
 [-1.3932254]]


### Layer Function Shortcuts

In [51]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x:[[1, 2, 3], [4, 5, 6],[7, 8, 9]]}))

[[-1.7432835]
 [-4.4192176]
 [-7.095151 ]]


## Feature Columns

In [0]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department' : ['sports', 'sports', 'gardening', 'sports']    
}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [    
    tf.feature_column.numeric_column('sales'),    
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

In [0]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess.run((var_init, table_init))

In [67]:
print(sess.run(inputs))
sess.close()

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 1.  0.  9.]]


## Training

### Define the data

In [0]:
x = tf.constant([[1], [2], [3], [4]], dtype = tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype = tf.float32)

### Define the model

In [0]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

In [85]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[-1.5702946]
 [-3.1405892]
 [-4.710884 ]
 [-6.2811785]]


### Loss

In [86]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

6.290743


### Training

In [0]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [88]:
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

6.290743
4.391176
3.0729523
2.1581085
1.5231628
1.0824336
0.7764689
0.56401426
0.4164455
0.31390074
0.24259807
0.19297434
0.15839416
0.13425326
0.11735688
0.105488114
0.09710881
0.09115164
0.08687595
0.08376787
0.08147083
0.07973738
0.07839582
0.07732703
0.076448314
0.075702325
0.07504923
0.07446141
0.07391971
0.07341078
0.07292539
0.07245715
0.07200154
0.07155554
0.07111694
0.070684284
0.07025646
0.06983281
0.069412805
0.068996064
0.06858234
0.06817147
0.0677633
0.06735775
0.06695476
0.06655423
0.06615618
0.065760545
0.065367326
0.06497644
0.064587936
0.064201765
0.06381789
0.06343633
0.06305704
0.06268002
0.062305287
0.06193274
0.061562464
0.061194386
0.060828518
0.060464837
0.060103312
0.059743993
0.05938677
0.059031717
0.058678776
0.058327943
0.05797921
0.05763258
0.05728799
0.056945477
0.056605004
0.05626657
0.055930167
0.055595767
0.055263363
0.05493296
0.05460453
0.054278053
0.053953543
0.05363096
0.053310297
0.052991573
0.052674744
0.052359823
0.052046776
0.05173559
0.051426265

In [89]:
print(sess.run(y_pred))
writer = tf.summary.FileWriter('/tmp/log')
writer.add_graph(tf.get_default_graph())
sess.close()

[[-0.3532285]
 [-1.1711634]
 [-1.9890983]
 [-2.8070333]]


# Stopping TensorBoard

In [0]:
def checkTensorBoardStatus():
  ps = !ps -ax
  is_tensorboard_running = len([f for f in ps if "tensorboard" in f ]) > 0

  is_ngrok_running = len([f for f in ps if "ngrok" in f ]) > 0
  print("tensorboard={}, ngrok={}".format(is_tensorboard_running, is_ngrok_running))


In [116]:
#Get pid of both processes
for f in ps:
  if "tensorboard" in f:
    print(f)
  if "ngrok" in f:
    print(f)

  192 ?        Sl     0:03 /usr/bin/python2 /usr/local/bin/tensorboard --logdir /tmp/log --host 0.0.0.0 --port 6006
  250 ?        Sl     0:23 ./ngrok http 6006


In [117]:
#kill ngrok
checkTensorBoardStatus()
!kill 250
!kill 192
checkTensorBoardStatus()

tensorboard=True, ngrok=True
tensorboard=False, ngrok=False
